# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

### My response
***

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
data = data[['race','call']]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 2 columns):
race    4870 non-null object
call    4870 non-null float32
dtypes: float32(1), object(1)
memory usage: 95.1+ KB


In [5]:
data.race.value_counts()

w    2435
b    2435
Name: race, dtype: int64

In [6]:
data.groupby(['race','call']).size().unstack()

call,0.0,1.0
race,,
b,2278,157
w,2200,235


In [7]:
data.groupby(['race','call']).size().apply(lambda a: a/2435.).unstack()

call,0.0,1.0
race,,
b,0.935524,0.064476
w,0.903491,0.096509


### 1. What test is appropriate for this problem? Does CLT apply?

The appropriate test is a two sample z-test for the difference in proportions test with independent samples.  We also assume that the sampling was random.  The CLT does apply in this case.

### 2. What are the null and alternative hypotheses?

We will conduct a two-sided test.  The null hypothesis is that there is no difference between the two population proportions.  The alternative hypothesis is that there is a difference between the two population proportions.

### 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [9]:
# Compute the callback rates for both races
n_b = len(data[data.race=='b'])
n_w = len(data[data.race=='w'])
c_b = sum(data[data.race=='b'].call)
c_w = sum(data[data.race=='w'].call)
p_b = c_b / n_b
p_w = c_w / n_w
print('Callback rates for blacks & whites are: {:1.3f}, {:1.3f}'.format(p_b,p_w))

Callback rates for blacks & whites are: 0.064, 0.097


In [10]:
# Compute the standard error for the difference between callback rates
se = np.sqrt((p_b*(1-p_b)/n_b)+(p_w*(1-p_w)/n_w))
print('Standard error for the difference between callback rates: {:1.4f}'.format(se))

Standard error for the difference between callback rates: 0.0078


In [11]:
# Find 95% and 99% confidence intervals
crit = stats.norm.isf([0.025,0.005])
ci_95 = [p_b-p_w-crit[0]*se, p_b-p_w+crit[0]*se]
print("95% confidence interval: ({:2.3f}, {:2.3f})".format(ci_95[0],ci_95[1]))

95% confidence interval: (-0.047, -0.017)


In [12]:
ci_99 = [p_b-p_w-crit[1]*se, p_b-p_w+crit[1]*se]
print("99% confidence interval: ({:2.3f}, {:2.3f})".format(ci_99[0],ci_99[1]))

99% confidence interval: (-0.052, -0.012)


In [14]:
# compute the p-value of two-tailed test
z_stat = (p_b - p_w) / se
p_value = 2*stats.norm.cdf(z_stat)
print('The p-value for the test of equality of callback rates is: {:1.5f}'.format(p_value))

The p-value for the test of equality of callback rates is: 0.00004


### 4. Write a story describing the statistical significance in the context or the original problem.

Because the p-value is less than 0.01, we reject the null hypothesis.  This means that there is a significant difference between the callbacks that white-sounding names get and the callbacks that black-sounding names get.  In fact, the difference suggests that the callback rate for black-sounding names is 3.3% percentage points less than the callback rate for white-sounding names.  This is evidence that racial discrimination in the US labor market still exists.

### 5. Does your analysis mean that race/name is the most important factor in callback success?  Why or why not? If not, how would you amend your analysis?

Unconditional on all factors that could affect the callback rate, such as education, experience, relevant skills, etc., we find a significant difference.  Without controlling for these other factors, we could be picking up other effects that are the underlying driving forces for the difference.  A simple extension would be to control for these other factors using a logistic regression.